In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import classification_report , confusion_matrix
from sklearn.model_selection import train_test_split

import cv2
#from google.colab.patches import cv2_imshow
from PIL import Image 
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D , MaxPooling2D, Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet152
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

In [ ]:
# Create an ImageDataGenerator with scaling and data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Scale pixel values to the range [0, 1]
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Load and preprocess the images using the generator
batch_size = 256  # Adjust the batch size according to your needs
image_size = (224, 224)

train_data_generator = datagen.flow_from_directory(
    "/kaggle/input/cat-or-dog-image-classification/Train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    #subset='training',  # Specify 'training' to get the training set
    shuffle=True
)

In [ ]:
test_data_generator = datagen.flow_from_directory(
    "/kaggle/input/cat-or-dog-image-classification/Test",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    #subset='validation',  # Specify 'validation' to get the validation set
    shuffle=True
)

In [ ]:
train_data_generator.class_indices

In [ ]:
class_labels = ["Cat","Dog"]

In [ ]:
for img,label in train_data_generator:
    print(img.shape)
    print(label)
    break

In [ ]:
import random

Visualizing Training Images

In [ ]:
path = os.listdir("/kaggle/input/cat-or-dog-image-classification/Train/Cat")
plt.figure(figsize=(25,30))
i=1
for img in random.sample(path,30):
    image = Image.open("/kaggle/input/cat-or-dog-image-classification/Train/Cat/"+img)
    plt.subplot(6,5,i)
    plt.imshow(image)
    i+=1

In [ ]:
path = os.listdir("/kaggle/input/cat-or-dog-image-classification/Train/Dog")
plt.figure(figsize=(25,30))
i=1
for img in random.sample(path,30):
    image = Image.open("/kaggle/input/cat-or-dog-image-classification/Train/Dog/"+img)
    plt.subplot(6,5,i)
    plt.imshow(image)
    i+=1

MobileNet Model Refinement

In [ ]:
path = "/kaggle/input/mobilenet-v2/tensorflow2/100-224-classification/2"

model_hub = hub.KerasLayer(path, input_shape=(224,224,3), trainable=False)

num_class = 1
model = Sequential()
model.add(model_hub)
model.add(Dense(1000,activation='relu'))
model.add(Dropout(0.3))
# model.add(Dense(1000,activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(units=num_class, activation="sigmoid"))


model.compile(optimizer="adam",loss="binary_crossentropy",
             metrics=["accuracy"])


model.summary()

In [ ]:
# Train the model on your new dataset
history = model.fit(train_data_generator, epochs=2,
            batch_size=256, validation_data = test_data_generator)

Evaluating the Model on Training and Test Data

In [ ]:
loss, acc = model.evaluate(train_data_generator)
print("Accuracy On Train Data:",acc)
loss, acc = model.evaluate(test_data_generator)
print("Accuracy On Test Data:",acc)

Model Inference

In [ ]:
y_pred = model.predict(test_data_generator)
y_pred = [np.argmax(i) for i in y_pred]

y_pred[:10]

In [ ]:
target_label = test_data_generator.class_indices
target_label

In [ ]:
path = os.listdir("/kaggle/input/cat-or-dog-image-classification/Test")
test_data = []
label = []
for i in path:
    #print(i)
    path = os.path.join("/kaggle/input/cat-or-dog-image-classification/Test",i)
    #print(path)
    for img in os.listdir(path):
        image = Image.open(path+"/"+img)
        image = image.convert("RGB")
        image = image.resize((224,224))
        image = np.array(image)
    
        test_data.append(image)
        label.append(target_label[i])

In [ ]:
x_test = np.array(test_data)
y_test = np.array(label) 


x_test_scaled = x_test/255
x_test_scaled.shape

Assess the Discrepancy Between Actual and Predicted Labels

In [ ]:
y_pred_test = model.predict(x_test_scaled)
y_pred_test = [1 if i>=0.5 else 0 for i in y_pred_test]
y_pred_test[:10]

In [ ]:
y_test[:10]

In [ ]:
plt.figure(figsize=(36,90))
for i in range(90):
    plt.subplot(15,6,i+1)
    plt.imshow(x_test[i])
    plt.title(f"Actual:{class_labels[y_test[i]]}\nPredicted:{class_labels[y_pred_test[i]]}")
    i+=1